This Model is just for test and its not the final Project the other Notebook is the complete project 

In [ ]:
!pip install datasets
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 14.9 MB/s 
     |████████████████████████████████| 212 kB 71.7 MB/s 
     |████████████████████████████████| 163 kB 64.6 MB/s 
     |████████████████████████████████| 115 kB 65.7 MB/s 
     |████████████████████████████████| 127 kB 72.2 MB/s 
     |████████████████████████████████| 115 kB 54.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 13.9 MB/s 
     |████████████████████████████████| 7.6 MB 51.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 14.3 MB/s 


In [ ]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

model_name = "persiannlp/mt5-base-parsinlu-opus-translation_fa_en"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/696 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [ ]:
model

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (w

In [ ]:
from google.colab import drive
drive.mount('content')

from datasets import load_dataset

train_path = 'content/MyDrive/trainset.csv'
dataset = load_dataset('csv', data_files={'train':train_path})
dataset

In [ ]:
def tokenizing(sample):
  return tokenizer(sample['Paersian_name'], padding='max_length')

def tokenizing_label(sample):
  return {'labels': tokenizer(sample['Pinglish_name'], padding='max_length')['input_ids']}

In [ ]:
dataset = dataset.map(tokenizing, batched=True)
dataset = dataset.map(tokenizing_label, batched=True)
dataset = dataset.remove_columns(['Paersian_name', 'Pinglish_name'])

In [ ]:
train_dataset = dataset['train'].shuffle(seed=10).select(range(4000))
eval_dataset = dataset['train'].shuffle(seed=10).select(range(4000, 4999))

In [ ]:
# !pip install jiwer

import numpy as np
from datasets import load_metric

metric = load_metric('wer')

def compute_metric(pred_eval):

  preds = pred_eval.predictions[0]
  label = pred_eval.label_ids

  preds = np.argmax(preds, axis=-1)
  labels = np.where(label != -100, label, 1)

  pred_str = tokenizer.batch_decode(preds)
  label_str = tokenizer.batch_decode(labels)

  return metric.compute(predictions=pred_str, references=label_str)

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="content/MyDrive/trainingModel",
    evaluation_strategy="steps",
    eval_steps = 200,
    logging_strategy="steps",
    logging_steps = 100,
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    report_to="tensorboard"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=compute_metric,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
model.save_pretrained("content/MyDrive/models2/mt5_username")
 
tokenizer.save_pretrained("content/MyDrive/models2/mt5_username")

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir 'content/MyDrive/trainingModel/runs'